# LESSON 2.4: Histogram Equalization and Histogram Matching
## Biomedical Image Processing Techniques

In this lesson:
- What is a histogram
- Histogram computation
- Histogram equalization algorithm
- Histogram matching (specification)
- Matching to custom distributions
- Applications in medical imaging

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 1. What is a Histogram?

A **histogram** shows the distribution of pixel intensities in an image.

- X-axis: Pixel intensity values (0-255 for 8-bit images)
- Y-axis: Number of pixels with that intensity

### Formula:
$$h(r_k) = n_k$$

Where:
- $r_k$ = intensity level k
- $n_k$ = number of pixels with intensity $r_k$

In [ ]:
def compute_histogram(image):
    """
    Compute histogram of a grayscale image.
    
    Parameters:
    - image: input grayscale image (uint8)
    
    Returns:
    - histogram: array of 256 values
    """
    histogram = np.zeros(256, dtype=np.int32)
    
    # Count pixels for each intensity
    for pixel_value in image.ravel():
        histogram[pixel_value] += 1
    
    return histogram

# Faster version using numpy
def compute_histogram_fast(image):
    """Fast histogram computation using numpy."""
    return np.bincount(image.ravel(), minlength=256)

In [ ]:
# Create sample images with different distributions
np.random.seed(42)

# Dark image (low intensity)
dark_image = np.random.randint(0, 80, (200, 200), dtype=np.uint8)

# Bright image (high intensity)
bright_image = np.random.randint(180, 255, (200, 200), dtype=np.uint8)

# Low contrast (narrow range)
low_contrast = np.random.randint(100, 150, (200, 200), dtype=np.uint8)

# Normal distribution
normal_image = np.clip(np.random.normal(128, 40, (200, 200)), 0, 255).astype(np.uint8)

In [ ]:
# Visualize images and their histograms
images = [dark_image, bright_image, low_contrast, normal_image]
titles = ['Dark Image', 'Bright Image', 'Low Contrast', 'Normal Distribution']

plt.figure(figsize=(16, 8))

for i, (img, title) in enumerate(zip(images, titles)):
    # Image
    plt.subplot(2, 4, i+1)
    plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    plt.title(title)
    plt.axis('off')
    
    # Histogram
    plt.subplot(2, 4, i+5)
    hist = compute_histogram_fast(img)
    plt.bar(range(256), hist, color='gray', width=1)
    plt.xlim([0, 255])
    plt.xlabel('Intensity')
    plt.ylabel('Count')
    plt.title(f'Histogram')

plt.suptitle('Images and Their Histograms', fontsize=14)
plt.tight_layout()
plt.show()

## 2. Normalized Histogram (PDF)

**Normalized histogram** = probability distribution function (PDF)

### Formula:
$$p(r_k) = \frac{n_k}{N}$$

Where:
- $N$ = total number of pixels
- $p(r_k)$ = probability of intensity $r_k$

In [ ]:
def normalized_histogram(image):
    """
    Compute normalized histogram (probability distribution).
    
    Returns:
    - pdf: probability for each intensity level
    """
    hist = compute_histogram_fast(image)
    total_pixels = image.size
    pdf = hist / total_pixels
    return pdf

In [ ]:
# Compare regular and normalized histograms
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
hist = compute_histogram_fast(normal_image)
plt.bar(range(256), hist, color='steelblue', width=1)
plt.xlabel('Intensity')
plt.ylabel('Pixel Count')
plt.title('Regular Histogram')
plt.xlim([0, 255])

plt.subplot(1, 2, 2)
pdf = normalized_histogram(normal_image)
plt.bar(range(256), pdf, color='coral', width=1)
plt.xlabel('Intensity')
plt.ylabel('Probability')
plt.title('Normalized Histogram (PDF)')
plt.xlim([0, 255])

plt.suptitle('Regular vs Normalized Histogram', fontsize=14)
plt.tight_layout()
plt.show()

print(f"Sum of PDF values: {pdf.sum():.4f} (should be 1.0)")

## 3. Cumulative Distribution Function (CDF)

**CDF** = cumulative sum of the PDF

### Formula:
$$CDF(r_k) = \sum_{j=0}^{k} p(r_j)$$

CDF is essential for histogram equalization!

In [ ]:
def compute_cdf(image):
    """
    Compute cumulative distribution function.
    
    Returns:
    - cdf: cumulative distribution for each intensity
    """
    pdf = normalized_histogram(image)
    cdf = np.cumsum(pdf)
    return cdf

In [ ]:
# Visualize PDF and CDF
plt.figure(figsize=(15, 5))

for i, (img, title) in enumerate(zip([dark_image, low_contrast, normal_image], 
                                      ['Dark Image', 'Low Contrast', 'Normal'])):
    plt.subplot(1, 3, i+1)
    
    pdf = normalized_histogram(img)
    cdf = compute_cdf(img)
    
    plt.bar(range(256), pdf, color='steelblue', width=1, alpha=0.7, label='PDF')
    plt.plot(range(256), cdf, 'r-', linewidth=2, label='CDF')
    
    plt.xlabel('Intensity')
    plt.ylabel('Probability / CDF')
    plt.title(title)
    plt.legend()
    plt.xlim([0, 255])
    plt.ylim([0, 1.1])

plt.suptitle('PDF and CDF Comparison', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Histogram Equalization

**Goal:** Transform image so that output histogram is approximately **uniform**.

### Algorithm:
1. Compute histogram $h(r_k)$
2. Compute normalized histogram (PDF) $p(r_k)$
3. Compute CDF $CDF(r_k)$
4. Transform: $s_k = (L-1) \cdot CDF(r_k)$

Where $L = 256$ for 8-bit images.

In [ ]:
def histogram_equalization(image):
    """
    Perform histogram equalization on a grayscale image.
    
    Parameters:
    - image: input grayscale image (uint8)
    
    Returns:
    - equalized: output image with equalized histogram
    - lookup_table: the transformation mapping
    """
    # Step 1: Compute histogram
    hist = compute_histogram_fast(image)
    
    # Step 2: Compute PDF
    pdf = hist / image.size
    
    # Step 3: Compute CDF
    cdf = np.cumsum(pdf)
    
    # Step 4: Create lookup table (transformation)
    # s_k = (L-1) * CDF(r_k)
    lookup_table = np.round(255 * cdf).astype(np.uint8)
    
    # Step 5: Apply transformation
    equalized = lookup_table[image]
    
    return equalized, lookup_table

In [ ]:
# Apply histogram equalization to different images
test_images = [dark_image, bright_image, low_contrast]
test_titles = ['Dark Image', 'Bright Image', 'Low Contrast']

plt.figure(figsize=(16, 12))

for i, (img, title) in enumerate(zip(test_images, test_titles)):
    equalized, lut = histogram_equalization(img)
    
    # Original image
    plt.subplot(3, 4, i*4 + 1)
    plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    plt.title(f'Original: {title}')
    plt.axis('off')
    
    # Original histogram
    plt.subplot(3, 4, i*4 + 2)
    plt.hist(img.ravel(), bins=256, range=(0, 256), color='steelblue', alpha=0.7)
    plt.xlim([0, 255])
    plt.title('Original Histogram')
    
    # Equalized image
    plt.subplot(3, 4, i*4 + 3)
    plt.imshow(equalized, cmap='gray', vmin=0, vmax=255)
    plt.title('Equalized')
    plt.axis('off')
    
    # Equalized histogram
    plt.subplot(3, 4, i*4 + 4)
    plt.hist(equalized.ravel(), bins=256, range=(0, 256), color='coral', alpha=0.7)
    plt.xlim([0, 255])
    plt.title('Equalized Histogram')

plt.suptitle('Histogram Equalization Results', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Understanding the Transformation Function

In [ ]:
# Visualize the transformation (lookup table)
plt.figure(figsize=(15, 5))

for i, (img, title) in enumerate(zip(test_images, test_titles)):
    _, lut = histogram_equalization(img)
    
    plt.subplot(1, 3, i+1)
    plt.plot(range(256), lut, 'b-', linewidth=2)
    plt.plot([0, 255], [0, 255], 'k--', alpha=0.5, label='Identity')
    plt.xlabel('Input Intensity')
    plt.ylabel('Output Intensity')
    plt.title(f'Transformation: {title}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xlim([0, 255])
    plt.ylim([0, 255])

plt.suptitle('Histogram Equalization Transformation Functions', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Step-by-Step Demonstration

In [ ]:
# Detailed step-by-step for dark image
img = dark_image

# Step 1: Histogram
hist = compute_histogram_fast(img)

# Step 2: PDF
pdf = hist / img.size

# Step 3: CDF
cdf = np.cumsum(pdf)

# Step 4: Lookup table
lut = np.round(255 * cdf).astype(np.uint8)

plt.figure(figsize=(16, 4))

plt.subplot(1, 4, 1)
plt.bar(range(256), hist, color='steelblue', width=1)
plt.title('Step 1: Histogram h(r)')
plt.xlabel('Intensity')
plt.xlim([0, 255])

plt.subplot(1, 4, 2)
plt.bar(range(256), pdf, color='coral', width=1)
plt.title('Step 2: PDF p(r)')
plt.xlabel('Intensity')
plt.xlim([0, 255])

plt.subplot(1, 4, 3)
plt.plot(range(256), cdf, 'g-', linewidth=2)
plt.title('Step 3: CDF')
plt.xlabel('Intensity')
plt.ylabel('Cumulative Probability')
plt.xlim([0, 255])
plt.ylim([0, 1])
plt.grid(True, alpha=0.3)

plt.subplot(1, 4, 4)
plt.plot(range(256), lut, 'r-', linewidth=2)
plt.title('Step 4: Lookup Table\ns = 255 * CDF(r)')
plt.xlabel('Input')
plt.ylabel('Output')
plt.xlim([0, 255])
plt.ylim([0, 255])
plt.grid(True, alpha=0.3)

plt.suptitle('Histogram Equalization: Step by Step', fontsize=14)
plt.tight_layout()
plt.show()

## 7. Medical Imaging Application: X-Ray Enhancement

In [ ]:
# Simulate a low-contrast X-ray image
np.random.seed(456)

# Create base structure (simulating bone/tissue)
xray = np.random.randint(60, 100, (256, 256), dtype=np.uint8)

# Add "bone" structures (slightly brighter)
xray[50:200, 100:120] = np.random.randint(100, 130, (150, 20), dtype=np.uint8)  # Spine
xray[80:120, 60:180] = np.random.randint(90, 120, (40, 120), dtype=np.uint8)   # Ribs
xray[140:180, 60:180] = np.random.randint(90, 120, (40, 120), dtype=np.uint8)  # Ribs

# Add some darker areas (lungs)
xray[70:190, 40:90] = np.random.randint(40, 70, (120, 50), dtype=np.uint8)
xray[70:190, 140:190] = np.random.randint(40, 70, (120, 50), dtype=np.uint8)

# Add Gaussian blur effect (simulate real X-ray)
from scipy.ndimage import gaussian_filter
xray = gaussian_filter(xray, sigma=2).astype(np.uint8)

In [ ]:
# Apply histogram equalization
xray_enhanced, _ = histogram_equalization(xray)

plt.figure(figsize=(14, 5))

plt.subplot(1, 3, 1)
plt.imshow(xray, cmap='gray')
plt.title('Original X-Ray\n(Low Contrast)')
plt.axis('off')
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(xray_enhanced, cmap='gray')
plt.title('After Histogram Equalization\n(Enhanced Contrast)')
plt.axis('off')
plt.colorbar()

plt.subplot(1, 3, 3)
plt.hist(xray.ravel(), bins=256, range=(0, 256), alpha=0.5, label='Original', color='blue')
plt.hist(xray_enhanced.ravel(), bins=256, range=(0, 256), alpha=0.5, label='Equalized', color='red')
plt.xlabel('Intensity')
plt.ylabel('Count')
plt.title('Histogram Comparison')
plt.legend()
plt.xlim([0, 255])

plt.suptitle('X-Ray Image Enhancement', fontsize=14)
plt.tight_layout()
plt.show()

## 8. Histogram Matching (Specification)

**Goal:** Transform image so that its histogram matches a **specified target** distribution.

Unlike equalization (which always targets uniform), matching can target **ANY** distribution.

### Algorithm:
1. Compute CDF of source image: $CDF_{src}$
2. Compute CDF of reference image: $CDF_{ref}$
3. For each source intensity $s$:
   - Find intensity $r$ where $CDF_{ref}(r) \approx CDF_{src}(s)$
   - Map: $s \rightarrow r$

### Key Insight:
Histogram equalization is a **special case** of histogram matching where the target is a uniform distribution.

In [ ]:
def histogram_matching(source, reference):
    """
    Match the histogram of a source image to a reference image.
    
    Parameters:
    - source: input image to be transformed (uint8)
    - reference: target image whose histogram we want to match (uint8)
    
    Returns:
    - matched: output image with matched histogram (uint8)
    - lookup_table: the transformation mapping (256 values)
    """
    # Step 1: Compute CDF of source
    src_hist = compute_histogram_fast(source)
    src_cdf = np.cumsum(src_hist / source.size)
    
    # Step 2: Compute CDF of reference
    ref_hist = compute_histogram_fast(reference)
    ref_cdf = np.cumsum(ref_hist / reference.size)
    
    # Step 3: Create lookup table
    # For each source intensity, find the closest matching reference intensity
    lookup_table = np.zeros(256, dtype=np.uint8)
    
    for src_val in range(256):
        # Find the reference intensity whose CDF value is closest
        diff = np.abs(ref_cdf - src_cdf[src_val])
        lookup_table[src_val] = np.argmin(diff)
    
    # Step 4: Apply transformation
    matched = lookup_table[source]
    
    return matched, lookup_table

In [ ]:
# Example 1: Match dark image to normal distribution
matched_dark, lut_dark = histogram_matching(dark_image, normal_image)

plt.figure(figsize=(18, 10))

# Source
plt.subplot(2, 3, 1)
plt.imshow(dark_image, cmap='gray', vmin=0, vmax=255)
plt.title('Source: Dark Image')
plt.axis('off')

plt.subplot(2, 3, 4)
plt.hist(dark_image.ravel(), bins=256, range=(0, 256), color='steelblue', alpha=0.7)
plt.xlim([0, 255])
plt.title('Source Histogram')
plt.xlabel('Intensity')

# Reference
plt.subplot(2, 3, 2)
plt.imshow(normal_image, cmap='gray', vmin=0, vmax=255)
plt.title('Reference: Normal Distribution')
plt.axis('off')

plt.subplot(2, 3, 5)
plt.hist(normal_image.ravel(), bins=256, range=(0, 256), color='forestgreen', alpha=0.7)
plt.xlim([0, 255])
plt.title('Reference Histogram')
plt.xlabel('Intensity')

# Result
plt.subplot(2, 3, 3)
plt.imshow(matched_dark, cmap='gray', vmin=0, vmax=255)
plt.title('Result: Matched Image')
plt.axis('off')

plt.subplot(2, 3, 6)
plt.hist(matched_dark.ravel(), bins=256, range=(0, 256), color='coral', alpha=0.7)
plt.xlim([0, 255])
plt.title('Matched Histogram')
plt.xlabel('Intensity')

plt.suptitle('Histogram Matching: Dark Image -> Normal Distribution', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Example 2: Match low contrast image to bright image
matched_low, lut_low = histogram_matching(low_contrast, bright_image)

plt.figure(figsize=(18, 10))

# Source
plt.subplot(2, 3, 1)
plt.imshow(low_contrast, cmap='gray', vmin=0, vmax=255)
plt.title('Source: Low Contrast')
plt.axis('off')

plt.subplot(2, 3, 4)
plt.hist(low_contrast.ravel(), bins=256, range=(0, 256), color='steelblue', alpha=0.7)
plt.xlim([0, 255])
plt.title('Source Histogram')
plt.xlabel('Intensity')

# Reference
plt.subplot(2, 3, 2)
plt.imshow(bright_image, cmap='gray', vmin=0, vmax=255)
plt.title('Reference: Bright Image')
plt.axis('off')

plt.subplot(2, 3, 5)
plt.hist(bright_image.ravel(), bins=256, range=(0, 256), color='forestgreen', alpha=0.7)
plt.xlim([0, 255])
plt.title('Reference Histogram')
plt.xlabel('Intensity')

# Result
plt.subplot(2, 3, 3)
plt.imshow(matched_low, cmap='gray', vmin=0, vmax=255)
plt.title('Result: Matched Image')
plt.axis('off')

plt.subplot(2, 3, 6)
plt.hist(matched_low.ravel(), bins=256, range=(0, 256), color='coral', alpha=0.7)
plt.xlim([0, 255])
plt.title('Matched Histogram')
plt.xlabel('Intensity')

plt.suptitle('Histogram Matching: Low Contrast -> Bright Image', fontsize=14)
plt.tight_layout()
plt.show()

## 9. Matching Transformation and CDF Comparison

In [ ]:
# Visualize transformation functions and CDF comparison
plt.figure(figsize=(15, 5))

# Transformation for dark -> normal
plt.subplot(1, 3, 1)
plt.plot(range(256), lut_dark, 'b-', linewidth=2)
plt.plot([0, 255], [0, 255], 'k--', alpha=0.5, label='Identity')
plt.xlabel('Input Intensity')
plt.ylabel('Output Intensity')
plt.title('Transformation: Dark -> Normal')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim([0, 255])
plt.ylim([0, 255])

# Transformation for low contrast -> bright
plt.subplot(1, 3, 2)
plt.plot(range(256), lut_low, 'r-', linewidth=2)
plt.plot([0, 255], [0, 255], 'k--', alpha=0.5, label='Identity')
plt.xlabel('Input Intensity')
plt.ylabel('Output Intensity')
plt.title('Transformation: Low Contrast -> Bright')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim([0, 255])
plt.ylim([0, 255])

# CDF comparison: source, reference, and matched result
plt.subplot(1, 3, 3)
cdf_dark = compute_cdf(dark_image)
cdf_normal = compute_cdf(normal_image)
cdf_matched = compute_cdf(matched_dark)
plt.plot(range(256), cdf_dark, 'b-', linewidth=2, label='Source (Dark)')
plt.plot(range(256), cdf_normal, 'g-', linewidth=2, label='Reference (Normal)')
plt.plot(range(256), cdf_matched, 'r--', linewidth=2, label='Matched Result')
plt.xlabel('Intensity')
plt.ylabel('CDF')
plt.title('CDF Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim([0, 255])
plt.ylim([0, 1.05])

plt.suptitle('Histogram Matching: Transformations and CDF', fontsize=14)
plt.tight_layout()
plt.show()

## 10. Matching to Custom Distributions

Histogram matching is not limited to real reference images. You can specify **any target distribution**: Gaussian, bimodal, exponential, or any custom shape.

In [ ]:
def match_to_distribution(source, target_pdf):
    """
    Match the histogram of a source image to a target PDF.

    Parameters:
    - source: input image (uint8)
    - target_pdf: desired probability distribution (256 values, sums to 1)

    Returns:
    - matched: output image (uint8)
    """
    # Source CDF
    src_hist = compute_histogram_fast(source)
    src_cdf = np.cumsum(src_hist / source.size)

    # Target CDF
    target_cdf = np.cumsum(target_pdf)

    # Create lookup table
    lookup_table = np.zeros(256, dtype=np.uint8)
    for s in range(256):
        diff = np.abs(target_cdf - src_cdf[s])
        lookup_table[s] = np.argmin(diff)

    return lookup_table[source]

# Create custom target distributions
x = np.arange(256)

# Gaussian distribution centered at 128
gaussian_pdf = np.exp(-0.5 * ((x - 128) / 30) ** 2)
gaussian_pdf = gaussian_pdf / gaussian_pdf.sum()

# Bimodal distribution (two peaks)
bimodal_pdf = (np.exp(-0.5 * ((x - 80) / 20) ** 2) +
               np.exp(-0.5 * ((x - 180) / 20) ** 2))
bimodal_pdf = bimodal_pdf / bimodal_pdf.sum()

# Exponential distribution
exp_pdf = np.exp(-x / 50.0)
exp_pdf = exp_pdf / exp_pdf.sum()

# Apply matching to low contrast image
matched_gauss = match_to_distribution(low_contrast, gaussian_pdf)
matched_bimodal = match_to_distribution(low_contrast, bimodal_pdf)
matched_exp = match_to_distribution(low_contrast, exp_pdf)

plt.figure(figsize=(16, 12))

# Row 1: Source
plt.subplot(4, 3, 1)
plt.imshow(low_contrast, cmap='gray', vmin=0, vmax=255)
plt.title('Source: Low Contrast')
plt.axis('off')

plt.subplot(4, 3, 2)
plt.hist(low_contrast.ravel(), bins=256, range=(0, 256), color='steelblue', alpha=0.7)
plt.xlim([0, 255])
plt.title('Source Histogram')

plt.subplot(4, 3, 3)
plt.axis('off')
plt.text(0.5, 0.5, 'Target distributions\nshown below', ha='center', va='center',
         fontsize=12, style='italic', transform=plt.gca().transAxes)

# Row 2: Gaussian target
plt.subplot(4, 3, 4)
plt.imshow(matched_gauss, cmap='gray', vmin=0, vmax=255)
plt.title('Matched: Gaussian')
plt.axis('off')

plt.subplot(4, 3, 5)
plt.hist(matched_gauss.ravel(), bins=256, range=(0, 256), color='coral', alpha=0.7)
plt.xlim([0, 255])
plt.title('Result Histogram')

plt.subplot(4, 3, 6)
plt.bar(x, gaussian_pdf, color='forestgreen', width=1)
plt.xlim([0, 255])
plt.title('Target: Gaussian PDF')

# Row 3: Bimodal target
plt.subplot(4, 3, 7)
plt.imshow(matched_bimodal, cmap='gray', vmin=0, vmax=255)
plt.title('Matched: Bimodal')
plt.axis('off')

plt.subplot(4, 3, 8)
plt.hist(matched_bimodal.ravel(), bins=256, range=(0, 256), color='coral', alpha=0.7)
plt.xlim([0, 255])
plt.title('Result Histogram')

plt.subplot(4, 3, 9)
plt.bar(x, bimodal_pdf, color='forestgreen', width=1)
plt.xlim([0, 255])
plt.title('Target: Bimodal PDF')

# Row 4: Exponential target
plt.subplot(4, 3, 10)
plt.imshow(matched_exp, cmap='gray', vmin=0, vmax=255)
plt.title('Matched: Exponential')
plt.axis('off')

plt.subplot(4, 3, 11)
plt.hist(matched_exp.ravel(), bins=256, range=(0, 256), color='coral', alpha=0.7)
plt.xlim([0, 255])
plt.title('Result Histogram')

plt.subplot(4, 3, 12)
plt.bar(x, exp_pdf, color='forestgreen', width=1)
plt.xlim([0, 255])
plt.title('Target: Exponential PDF')

plt.suptitle('Histogram Matching to Custom Distributions', fontsize=14)
plt.tight_layout()
plt.show()

## 11. Equalization vs Matching: Comparison

| Feature | Histogram Equalization | Histogram Matching |
|---|---|---|
| **Target** | Uniform distribution | Any specified distribution |
| **Reference needed?** | No | Yes (image or PDF) |
| **Control** | Automatic | User-controlled |
| **Use case** | Maximize contrast | Match specific appearance |
| **Risk** | Can over-enhance | More predictable output |

In [ ]:
# Visual comparison: Original vs Equalized vs Matched
equalized_dark, _ = histogram_equalization(dark_image)
matched_to_normal, _ = histogram_matching(dark_image, normal_image)

plt.figure(figsize=(14, 5))

plt.subplot(1, 3, 1)
plt.imshow(dark_image, cmap='gray', vmin=0, vmax=255)
plt.title('Original (Dark)')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(equalized_dark, cmap='gray', vmin=0, vmax=255)
plt.title('Equalized\n(Uniform Target)')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(matched_to_normal, cmap='gray', vmin=0, vmax=255)
plt.title('Matched\n(Gaussian Target)')
plt.axis('off')

plt.suptitle('Equalization vs Matching: Visual Comparison', fontsize=14)
plt.tight_layout()
plt.show()

## Summary

What we learned:

1. **Histogram**: Distribution of pixel intensities $h(r_k) = n_k$

2. **PDF**: Normalized histogram $p(r_k) = n_k / N$

3. **CDF**: Cumulative distribution $CDF(r_k) = \sum p(r_j)$

4. **Histogram Equalization**:
   - Targets **uniform** distribution
   - Formula: $s = 255 \cdot CDF(r)$
   - Automatically enhances contrast

5. **Histogram Matching (Specification)**:
   - Targets **any specified** distribution
   - Maps via CDF of source and reference
   - More control than equalization
   - Can match to real images or synthetic PDFs

6. **Key Difference**:
   - Equalization = special case of matching (uniform target)
   - Matching = general case (any target)

7. **Medical Imaging Applications**:
   - Enhance low-contrast X-ray, CT, MRI images
   - Standardize images across different scanners
   - Pre-processing for automated analysis